In [24]:
import urllib.request, json 
import requests
import pandas as pd


In [25]:
with urllib.request.urlopen("https://assay-screening.herokuapp.com/api/well-aggregate/") as url:
    well_by_id_assay = json.loads(url.read().decode())
    master_table = []
    summary_table = []
for record in well_by_id_assay:
    experiment_id = record['experiment_id']
    qpcr_plate_id = record['qpcr_plate_id']
    wells =json.dumps(record['wells'].split(","))    
    _url = "https://assay-screening.herokuapp.com/api/well-results/?expt=%s&plate_id=%s&wells=%s"%(experiment_id,qpcr_plate_id,wells)    
    response =json.loads(requests.get(_url).text)
    master_table = master_table + response['master_table']
    summary_table = summary_table +response['summary_table']

In [66]:
def dataframe_list_to_string(df):
    list_cols = ['ID Assay Conc.','ID Assay Name','ID Human Conc.','ID Human Name',
               'ID Template Conc.','ID Template Name','PA Assay Conc.','PA Assay Name',
               'PA Human Conc.','PA Human Name','PA Template Conc.','PA Template Name']
    for column in list_cols:
            df[column] = df[column].apply(list_to_string)
    return df

In [67]:
def list_to_string(value):
    if not isinstance(value,float):
        return "".join(str(e) for e in value)
    return None

In [46]:
def find_sample(row):    
    name = ''
    if row['ID Template Conc.']:
        name += '%s cp '%row['ID Template Conc.']
    if row['ID Human Conc.']:
        name += '%s ng HgDNA '%row['ID Human Conc.']
    if row['PA Template Conc.']:
        name += 'PA %s cp '%row['PA Template Conc.']
    if row['PA Human Conc.']:
        if 'PA' not in name:
            name += 'PA %s ng HgDNA '%row['PA Human Conc.']
        else:
            name += '%s ng HgDNA '%row['PA Human Conc.']
    if name =='':
        name = 'NTC'
    return name
    

In [33]:
master_df = dataframe_list_to_string(pd.DataFrame(master_table))

In [139]:
summary_df = dataframe_list_to_string(pd.DataFrame(summary_table))

In [35]:
master_df[['qPCR Plate','qPCR Well','LC Plate','LC Well','ID Assay Name','ID Template Name',
    'ID Template Conc.','ID Human Conc.','PA Assay Name','PA Template Name',
    'PA Template Conc.','PA Human Conc.','Ct','Ct Call','Tm1', 'Tm2', 'Tm3', 'Tm4','Tm Specif','Tm NS',
    'Tm PD']].to_csv('master.csv')

In [37]:
summary_df[['qPCR Plate','LC Plate','ID Assay Name','ID Template Name',
    'ID Template Conc.','ID Human Conc.','PA Assay Name','PA Template Name',
    'PA Template Conc.','PA Human Conc.','Reps', 'Max Ct','Min Ct','Mean Ct',
    'Mean ∆NTC Ct','#Ct Pos', 'Max Tm1','Min Tm1','Mean Tm1','#Tm Specif','#Tm NS', '#Tm PD',
   'Mean Specif ng/ul','Mean NS ng/ul', 'Mean PD ng/ul']].to_csv('summary.csv')

In [140]:
summary_df['sample'] = summary_df.apply(find_sample,axis=1)

In [147]:
rows = ['NTC','50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','1000.0 cp ',
             '50.0 ng HgDNA ','100.0 cp ','10000.0 cp ','500.0 ng HgDNA ']
ind = ['Expt','qPCR Plate','Reps']
dep = ['#Ct Pos', '#Tm Specif', '#Tm PD', '#Tm NS', 'Mean Ct', 'Min Ct',
           'Max Ct', 'Min Tm1', 'Max Tm1', 'Mean Specif ng/ul','Mean NS ng/ul', 'Mean PD ng/ul'] 

def summary_by_sample(df,rows,ind,dep):

    df = df[df['qPCR Plate']!='A81_E214_1_ID'] 

    frames = []
# ['Expt','plate_id','Group','templates']
    for n, grp in df.groupby(['qPCR Plate','ID Assay Name','ID Template Name']):    
        a = pd.DataFrame(grp.set_index('sample').loc[rows].stack())
        a = a.sort_index(level=[0, 1])
        a = a.sort_index()
        b = a.loc[idx['NTC', ind], :]
        b.index = pd.MultiIndex.from_product([['Common'], b.index.get_level_values(1)])
        c = a.loc[idx[:, dep], :]

        df = pd.concat([b, c])
        df.columns = pd.MultiIndex.from_tuples([n])
        #df = df.rename(columns={0: n[2]})
        frames.append(df)
    return frames

In [148]:
def removeErrFrames(frames):
    expt_list=[]
    new_df=frames[0].T
    new_df=new_df.drop(new_df.index[0])
    for i in range(1,len(frames)):         
        try:
            new_df=new_df.append(frames[i].T)
        except:
            expt_list.append(i)
    for index in sorted(expt_list, reverse=True):
        print(frames[index])
        del frames[index]
    return frames
    


In [149]:
frames=removeErrFrames(summary_by_sample(summary_df,rows,ind,dep))
frames_df = pd.concat(frames, axis=1).reindex(rows, level=0)
frames_df = frames_df.sort_index().T

c:\users\prasanna.gnanaraj\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  app.launch_new_instance()


In [150]:
frames_df = frames_df.sort_index(axis=1, level=[0, 1])

In [151]:
frames_df.to_csv('group_by_columns.csv')